In [ ]:
from ultralytics import YOLO

# create trainer from config + weights (transfer learning)
model = YOLO("yolov8m_P2.yaml")          # architecture
# load weights (backbone will be matched automatically)
model.load_weights("yolov8m.pt")                # or model = YOLO("yolov8m.pt") then change .model?

# train
model.train(
    data="training/BDD100K.yaml",
    epochs=10,
    imgsz=1280,
    batch=4,
    device="cpu",           # or "cuda:0" or "cpu"
    optimizer="AdamW",
    lr0=1e-3,
    lrf=0.01,
    weight_decay=5e-4,
    amp=True
)


In [1]:
from ultralytics import YOLO

# ==============================
# CONFIGURATION
# ==============================
MODEL_CFG = "yolov8m_P2.yaml"        # your custom P2-head model
DATA_CFG = "training/BDD100K.yaml"              # dataset YAML
PRETRAINED_WEIGHTS = "/workspace/models/yolov8m.pt"           # pretrained model from Ultralytics
DEVICE = "cpu"                              # "cuda:0" for GPU, "mps" for MacBook, or "cpu"

# image + optimization settings
IMG_SIZE = 1280
EPOCHS_STAGE1 = 1   # warmup (frozen)
EPOCHS_STAGE2 = 9   # full fine-tune
BATCH_SIZE = 2       # for MacBook M2, adjust as needed
ACCUMULATE = 4       # simulate larger effective batch
AMP = True           # mixed precision

# optimizer + scheduler
OPTIMIZER = "AdamW"
LR0 = 1e-3
LRF = 0.01
MOMENTUM = 0.937
WEIGHT_DECAY = 5e-4
WARMUP_EPOCHS = 3.0
WARMUP_MOMENTUM = 0.8
WARMUP_BIAS_LR = 0.1

# loss weighting
BOX_GAIN = 7.5
CLS_GAIN = 1.0
DFL_GAIN = 1.5

# augmentations (Ultralytics built-ins)
AUGMENTATIONS = {
    "hsv_h": 0.015,
    "hsv_s": 0.7,
    "hsv_v": 0.4,
    "degrees": 0.0,
    "translate": 0.1,
    "scale": 0.5,
    "shear": 0.0,
    "perspective": 0.0,
    "flipud": 0.0,
    "fliplr": 0.5,
    "mosaic": 1.0,
    "mixup": 0.0,
    "copy_paste": 0.0,
}

# ==============================
# TRAINING PIPELINE
# ==============================
if __name__ == "__main__":
    # --- Stage 1: Freeze backbone (warmup) ---
    print("\n🔹 Stage 1: Warming up detection head (backbone frozen)...")

    model = YOLO(MODEL_CFG)
    model.load_weights(PRETRAINED_WEIGHTS)

    # Freeze all backbone layers (up to layer index 9 based on your YAML)
    for i, layer in enumerate(model.model):
        if i <= 9:  # backbone layers 0–9
            for p in layer.parameters():
                p.requires_grad = False

    model.train(
        data=DATA_CFG,
        epochs=EPOCHS_STAGE1,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        accumulate=ACCUMULATE,
        optimizer=OPTIMIZER,
        lr0=LR0 * 0.1,  # smaller LR for frozen stage
        lrf=LRF,
        momentum=MOMENTUM,
        weight_decay=WEIGHT_DECAY,
        warmup_epochs=WARMUP_EPOCHS,
        warmup_momentum=WARMUP_MOMENTUM,
        warmup_bias_lr=WARMUP_BIAS_LR,
        box=BOX_GAIN,
        cls=CLS_GAIN,
        dfl=DFL_GAIN,
        amp=AMP,
        device=DEVICE,
        **AUGMENTATIONS
    )

    # --- Stage 2: Unfreeze all layers (full training) ---
    print("\n🔹 Stage 2: Fine-tuning entire network...")

    # Unfreeze backbone
    for layer in model.model:
        for p in layer.parameters():
            p.requires_grad = True

    model.train(
        data=DATA_CFG,
        epochs=EPOCHS_STAGE2,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        accumulate=ACCUMULATE,
        optimizer=OPTIMIZER,
        lr0=LR0,
        lrf=LRF,
        momentum=MOMENTUM,
        weight_decay=WEIGHT_DECAY,
        warmup_epochs=WARMUP_EPOCHS,
        warmup_momentum=WARMUP_MOMENTUM,
        warmup_bias_lr=WARMUP_BIAS_LR,
        box=BOX_GAIN,
        cls=CLS_GAIN,
        dfl=DFL_GAIN,
        amp=AMP,
        device=DEVICE,
        resume=False,       # ensure fresh run
        **AUGMENTATIONS
    )

    print("\n✅ Training complete! Check your runs/detect/train*/ directory for logs and weights.")



🔹 Stage 1: Warming up detection head (backbone frozen)...


[W1020 12:10:25.120152312 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1020 12:10:25.181817521 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1020 12:10:25.201783687 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1020 12:10:25.251642521 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1020 12:10:25.271364729 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1020 12:10:25.311554271 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1020 12:10:25.336727437 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1020 12:10:25.338840521 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1020 12:10:25.342395146 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1020 12:10:25.345197437 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.


AttributeError: 'DetectionModel' object has no attribute 'load_weights'